<br>

<div align=center><font color=maroon size=6><b>TorchVision Object Detection Finetuning Tutorial</b></font></div>

<br>

<font size=4><b>References:</b></font>
1. Pytorch official tutorials: <a href="https://pytorch.org/tutorials/index.html" style="text-decoration:none;">WELCOME TO PYTORCH TUTORIALS</a>
    * Tutorials > <a href="https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html" style="text-decoration:none;">TorchVision Object Detection Finetuning Tutorial</a>

<br>
<br>
<br>

<div class="alert alert-block alert-success">

<font size=3 color=Green><b>TIP: </b></font>
<br><br>
<font color=black>
To get the most of this tutorial, we suggest using this <a href="https://colab.research.google.com/github/pytorch/tutorials/blob/gh-pages/_downloads/torchvision_finetuning_instance_segmentation.ipynb" style="text-decoration:none;color:maroon;font-size:120%;">Colab Version</a>. This will allow you to experiment with the information presented below.
</font>
</div>

<br>

For this tutorial, we will be finetuning a pre-trained <a href="https://arxiv.org/abs/1703.06870" style="text-decoration:none;color:maroon;font-size:120%;">Mask R-CNN</a> model in the <a href="https://www.cis.upenn.edu/~jshi/ped_html/" style="text-decoration:none;color:maroon;font-size:120%;">Penn-Fudan Database for Pedestrian Detection and Segmentation</a>. It contains 170 images with 345 instances of pedestrians, and we will use it to illustrate how to use the new features in torchvision in order to train an instance segmentation model on a custom dataset.

<br>
<br>

## **Defining the Dataset** <a href="https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html#defining-the-dataset" style="text-decoration:none;font-size:70%">[link]</a>

<font size=3 color=maroon>The reference scripts for training `object detection, instance segmentation and person keypoint detection` allows for easily supporting adding new custom datasets. The dataset should inherit from the standard ***`torch.utils.data.Dataset`*** class, and implement **\_\_len\_\_** and **\_\_getitem\_\_**.</font>


* ***image***: a PIL Image of size `(H, W)`.
* ***target***: a dict containing the following fields.
    * **`boxes`**` (FloatTensor[N, 4])`: the coordinates of the `N` bounding boxes in `[x0, y0, x1, y1]` format, ranging from `0` to `W` and `0` to `H`
    * **`labels`**` (Int64Tensor[N])`: the label for each bounding box. `0` represents always the background class.
    * **`image_id`**` (Int64Tensor[1])`: an image identifier. It should be unique between all the images in the dataset, and is used during evaluation.
    * **`area`**` (Tensor[N])`: The area of the bounding box. This is used during evaluation with the COCO metric, to separate the metric scores between small, medium and large boxes.
    * **`iscrowd`**` (UInt8Tensor[N])`: instances with iscrowd=True will be ignored during evaluation.
    * (optionally) **`masks`**` (UInt8Tensor[N, H, W])`: The segmentation masks for each one of the objects
    * (optionally) **`keypoints`**` (FloatTensor[N, K, 3])`: For each one of the N objects, it contains the K keypoints in `[x, y, visibility]` format, defining the object. `visibility=0` means that the keypoint is not visible. **Note that** for data augmentation, the notion of flipping a keypoint is dependent on the data representation, and you should probably adapt ***`references/detection/transforms.py`*** (<a href="https://github.com/pytorch/vision/blob/main/references/detection/transforms.py" style="text-decoration:none;font-size:100%;">link</a>) for your new keypoint representation

<font size=3 color=maroon>If your model returns the above methods, they will make it work for both training and evaluation, and will use the evaluation scripts from ***pycocotools*** which can be installed with `pip install pycocotools`.</font>

<div class="alert alert-block alert-info">

<font size=3 color=red><b>NOTE: </b></font>

For Windows, please install pycocotools from <a href="https://github.com/gautamchitnis/cocoapi" style="text-decoration:none;color:maroon;font-size:120%;">gautamchitnis</a> with command:
<br>
<br>
<font color=black>**pip install** git+https://github.com/gautamchitnis/cocoapi.git@cocodataset-master#subdirectory=PythonAPI</font><br><br>
(可能需要先安装 Cython：pip install Cython, 否则可能会报错。)
</div>

<br>

<font color=maroon>One note on the **labels**</font>. The model considers class 0 as background. If your dataset does not contain the background class, you should not have 0 in your labels. For example, assuming you have just two classes, cat and dog, you can define 1 (not 0) to represent cats and 2 to represent dogs. So, for instance, if one of the images has both classes, your labels tensor should look like [1,2].

<font color=maroon>Additionally</font>, if you want to use **aspect ratio grouping** during training (so that each batch only contains images with similar aspect ratios), then it is recommended to also implement a get_height_and_width method, which returns the height and the width of the image. If this method is not provided, we query all elements of the dataset via ***`__getitem__`*** , which loads the image in memory and is slower than if a custom method is provided.

<br>

### <font color=magenta>Writing a custom dataset for PennFudan</font> <a href="https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html#writing-a-custom-dataset-for-pennfudan" style="text-decoration:none;font-size:70%">[link]</a>

Let’s write a dataset for the PennFudan dataset. After <a href="https://www.cis.upenn.edu/~jshi/ped_html/PennFudanPed.zip" style="text-decoration:none;color:maroon;font-size:120%;">downloading and extracting the zip file</a>, we have the following folder structure:

Here is one example of a pair of images and segmentation masks:

<img src="./images/tv_image01.png" width=600px>
<img src="./images/tv_image02.png" width=600px>

So each image has a corresponding segmentation mask, where each color correspond to a different instance. Let’s write a ***`torch.utils.data.Dataset`*** class for this dataset.

In [1]:
import os
import numpy as np
import torch
from PIL import Image

In [2]:
class PennFudanDataset(torch.utils.data.Dataset):
    def __init__(self, root, transforms):
        self.root = root
        self.transforms = transforms
        # load all image files, sorting them to
        # ensure that they are aligned
        self.imgs = list(sorted(os.listdir(os.path.join(root, "PNGImages"))))
        self.masks = list(sorted(os.listdir(os.path.join(root, "PedMasks"))))

    def __getitem__(self, idx):
        # load images and masks
        img_path = os.path.join(self.root, "PNGImages", self.imgs[idx])
        mask_path = os.path.join(self.root, "PedMasks", self.masks[idx])
        img = Image.open(img_path).convert("RGB")
        # note that we haven't converted the mask to RGB,
        # because each color corresponds to a different instance
        # with 0 being background
        mask = Image.open(mask_path)
        # convert the PIL Image into a numpy array
        mask = np.array(mask)
        # instances are encoded as different colors
        obj_ids = np.unique(mask)
        # first id is the background, so remove it
        obj_ids = obj_ids[1:]

        # split the color-encoded mask into a set
        # of binary masks
        masks = mask == obj_ids[:, None, None]

        # get bounding box coordinates for each mask
        num_objs = len(obj_ids)
        boxes = []
        for i in range(num_objs):
            pos = np.where(masks[i])
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            boxes.append([xmin, ymin, xmax, ymax])

        # convert everything into a torch.Tensor
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # there is only one class
        labels = torch.ones((num_objs,), dtype=torch.int64)
        masks = torch.as_tensor(masks, dtype=torch.uint8)

        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        # suppose all instances are not crowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.imgs)

That’s all for the dataset. Now let’s define a model that can perform predictions on this dataset.

<br>
<br>
<br>

## <font color=magenta>**Defining your model**</font> <a href="https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html#defining-your-model" style="text-decoration:none;font-size:70%">[link]</a>

<font size=3>In this tutorial, we will be using <a href="https://arxiv.org/abs/1703.06870" style="text-decoration:none;color:maroon;font-size:120%;">Mask R-CNN</a>, which is based on top of <a href="https://arxiv.org/abs/1506.01497" style="text-decoration:none;color:maroon;font-size:120%;">Faster R-CNN</a>. Faster R-CNN is a model that predicts both bounding boxes and class scores for potential objects in the image.</font>

<img src="./images/tv_image03.png" width=600px>

<br>

<font size=3>Mask R-CNN adds an extra branch into Faster R-CNN, which also predicts segmentation masks for each instance.</font>

<img src="./images/tv_image04.png" width=600px>

<br>

<font size=3>There are two common situations where one might want to modify one of the available models in torchvision modelzoo. 
* The first is when we want to <font color=maroon>start from a pre-trained model, and just finetune the last layer</font>. 


* The other is when we want to <font color=maroon>replace the backbone of the model</font> with a different one (for faster predictions, for example).

    
Let’s go see how we would do one or another in the following sections.</font>


<br>
<br>

### <font color=magenta>1 - Finetuning from a pretrained model</font> <a href="https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html#finetuning-from-a-pretrained-model" style="text-decoration:none;font-size:70%">[link]</a>

Let’s suppose that you want to start from a model pre-trained on COCO and want to finetune it for your particular classes. Here is a possible way of doing it:


<font style="font-size:120%;LINE-HEIGHT: 30px">

```python
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# load a model pre-trained on COCO
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

# replace the classifier with a new one, that has num_classes which is user-defined
num_classes = 2     # 1 class (person) + background
    
# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features
    
# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
```

</font>

<br>
<br>

### <font color=magenta>2 - Modifying the model to add a different backbone</font> <a href="https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html#modifying-the-model-to-add-a-different-backbone" style="text-decoration:none;font-size:70%">[link]</a>

<font style="font-size:120%;LINE-HEIGHT: 30px">

```python
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

# load a pre-trained model for classification and return
# only the features
backbone = torchvision.models.mobilenet_v2(pretrained=True).features
# FasterRCNN needs to know the number of
# output channels in a backbone. For mobilenet_v2, it's 1280
# so we need to add it here
backbone.out_channels = 1280

# let's make the RPN generate 5 x 3 anchors per spatial
# location, with 5 different sizes and 3 different aspect
# ratios. We have a Tuple[Tuple[int]] because each feature
# map could potentially have different sizes and
# aspect ratios
anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),),
                                   aspect_ratios=((0.5, 1.0, 2.0),))

# let's define what are the feature maps that we will
# use to perform the region of interest cropping, as well as
# the size of the crop after rescaling.
# if your backbone returns a Tensor, featmap_names is expected to
# be [0]. More generally, the backbone should return an
# OrderedDict[Tensor], and in featmap_names you can choose which
# feature maps to use.
roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=['0'],
                                                output_size=7,
                                                sampling_ratio=2)

# put the pieces together inside a FasterRCNN model
model = FasterRCNN(backbone,
                   num_classes=2,
                   rpn_anchor_generator=anchor_generator,
                   box_roi_pool=roi_pooler)
```

</font>

<br>
<br>
<br>

## <font color=magenta>**An Instance segmentation model for PennFudan Dataset**</font> <a href="https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html#an-instance-segmentation-model-for-pennfudan-dataset" style="text-decoration:none;font-size:70%">[link]</a>

In our case, we want to fine-tune from a pre-trained model, given that our dataset is very small, so we will be following approach number 1.

Here we want to also compute the instance segmentation masks, so we will be using Mask R-CNN:

In [3]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor


def get_model_instance_segmentation(num_classes):
    # load an instance segmentation model pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model

That’s it, this will make `model` be ready to be trained and evaluated on your custom dataset.

<br>
<br>
<br>

## <font color=magenta>**Putting everything together**</font> <a href="https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html#putting-everything-together" style="text-decoration:none;font-size:70%">[link]</a>

In <a href="https://github.com/pytorch/vision/tree/main/references" style="text-decoration:none;color:maroon;font-size:120%;">references/detection/</a>, we have a number of helper functions to simplify training and evaluating detection models. Here, we will use `references/detection/engine.py`, `references/detection/utils.py` and `references/detection/transforms.py`. Just copy everything under `references/detection` to your folder and use them here.

`上述文件我下载在了路径：.\vision\references\detection\  之下`

In [6]:
'''
from vision.references.detection import transforms as T
from vision.references.detection import utils
from vision.references.detection import engine
'''
# 上述语句在导入 engine 时会报错：ModuleNotFoundError: No module named 'utils'

# 向 sys.path 中临时添加模块文件存储位置的完整路径
import sys
import os
cwd = os.getcwd()

# sys.path.append(os.path.join(cwd, "vision/references/detection/"))
# >>> D:\\KeepStudy\\0_Coding\\Pytorch\\Pytorch_Tutorials\\vision/references/detection/'

sys.path.append(os.path.join(cwd, "vision", "references", "detection"))
# sys.path     # 可见已经添加了上述路径

<br>

In [9]:
import transforms as T
import utils

In [10]:
def get_transform(train):
    transforms = []
    
    # 陈锐平修改
    # transforms.append(T.ToTensor())
    # AttributeError: module 'vision.references.detection.transforms' has no attribute 'ToTensor'
    try:
        transforms.append(T.ToTensor())
    except AttributeError:
        from torchvision.transforms import ToTensor
        transforms.append(ToTensor())
                    
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

<br>
<br>
<br>

## <font color=magenta>**Testing *forward()* method**</font> (Optional) <a href="https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html#testing-forward-method-optional" style="text-decoration:none;font-size:70%">[link]</a>

Before iterating over the dataset, it’s good to see what the model expects during training and inference time on sample data.

In [ ]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
dataset = PennFudanDataset('./data/PennFudanPed', get_transform(train=True))

data_loader = torch.utils.data.DataLoader(dataset, batch_size=2, 
                                          shuffle=True, 
                                          num_workers=4,
                                          collate_fn=utils.collate_fn)
# 这个 cell 的代码报错：
# RuntimeError: DataLoader worker (pid(s) 7924, 14544, 15588, 5232) exited unexpectedly


# For Training
images,targets = next(iter(data_loader))
images = list(image for image in images)
targets = [{k: v for k, v in t.items()} for t in targets]
output = model(images,targets)   # Returns losses and detections

# For inference
model.eval()
x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]
predictions = model(x)           # Returns predictions

<br>

Let’s now write the main function which performs the training and the validation:

In [18]:
from engine import train_one_epoch, evaluate
# import utils


def main():
    # train on the GPU or on the CPU, if a GPU is not available
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    # our dataset has two classes only - background and person
    num_classes = 2
    # use our dataset and defined transformations
    dataset = PennFudanDataset('./data/PennFudanPed', get_transform(train=True))
    dataset_test = PennFudanDataset('./data/PennFudanPed', get_transform(train=False))

    # split the dataset in train and test set
    indices = torch.randperm(len(dataset)).tolist()
    dataset = torch.utils.data.Subset(dataset, indices[:-50])
    dataset_test = torch.utils.data.Subset(dataset_test, indices[-50:])

    # define training and validation data loaders
    data_loader = torch.utils.data.DataLoader(
        dataset, batch_size=2, shuffle=True, num_workers=4,
        collate_fn=utils.collate_fn)

    data_loader_test = torch.utils.data.DataLoader(
        dataset_test, batch_size=1, shuffle=False, num_workers=4,
        collate_fn=utils.collate_fn)

    # get the model using our helper function
    model = get_model_instance_segmentation(num_classes)

    # move model to the right device
    model.to(device)

    # construct an optimizer
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=0.005,
                                momentum=0.9, weight_decay=0.0005)
    # and a learning rate scheduler
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                   step_size=3,
                                                   gamma=0.1)

    # let's train it for 10 epochs
    num_epochs = 10

    for epoch in range(num_epochs):
        # train for one epoch, printing every 10 iterations
        train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
        # update the learning rate
        lr_scheduler.step()
        # evaluate on the test dataset
        evaluate(model, data_loader_test, device=device)

    print("That's it!")

In [19]:
main()

Downloading: "https://download.pytorch.org/models/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth" to C:\Users\18617/.cache\torch\hub\checkpoints\maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth
100.0%


RuntimeError: DataLoader worker (pid(s) 19612, 3172, 17900, 18668) exited unexpectedly

<br>

You should get as output for the first epoch: `(详见原网址)`

So after one epoch of training, we obtain a COCO-style mAP of 60.6, and a mask mAP of 70.4.

After training for 10 epochs, I got the following metrics: `(详见原网址)`

<br>

But what do the predictions look like? Let’s take one image in the dataset and verify. `(详见原网址)`

The trained model predicts 9 instances of person in this image, let’s see a couple of them: `(详见原网址)`

The results look pretty good!

<br>
<br>
<br>

## **Wrapping up** <a href="https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html#wrapping-up" style="text-decoration:none;font-size:70%">[link]</a>

In this tutorial, you have learned how to create your own training pipeline for instance segmentation models, on a custom dataset. For that, you wrote a `torch.utils.data.Dataset` class that returns the **images** and the **ground truth boxes** and **segmentation masks**. You also leveraged a Mask R-CNN model pre-trained on COCO train2017 in order to perform transfer learning on this new dataset.

<font size=3>For a more complete example, which includes multi-machine / multi-gpu training, check <a href="https://github.com/pytorch/vision/tree/main/references/detection" style="text-decoration:none;color:maroon;font-size:120%;">references/detection/train.py</a>, which is present in the torchvision repo.

You can download a full source file for this tutorial <a href="https://pytorch.org/tutorials/_static/tv-training-code.py" style="text-decoration:none;color:maroon;font-size:120%;">here</a>.
</font>


<br>
<br>
<font size=3></font>

<a href="" style="text-decoration:none;font-size:70%">[link]</a>
<a href="" style="text-decoration:none;font-size:120%"></a>


<a href="" style="text-decoration:none;color:maroon;font-size:120%;"></a>

&emsp;&emsp;&emsp;&ensp;
<a href="" style="text-decoration:none;color:maroon;font-size:140%;font-weight:bold;"></a>


<font style="font-size:120%;LINE-HEIGHT: 30px">

```python

```

</font>

<font color=maroon>****</font>

<br>
<br>
<br>